In [1]:
import utils
import numpy as np

In [2]:
def region_growing(im: np.ndarray, seed_points: list, T: int) -> np.ndarray:
    """
    A region growing algorithm that segments an image into 1 or 0 (True or False).
    Uses an 8-connected Moore neighborhood and a threshold as the homogeneity criterion.
    
    Args:
        im (np.ndarray): Grayscale image in range [0, 255] (dtype=np.uint8)
        seed_points (list): List of [row, col] seed points
        T (int): Intensity threshold for homogeneity
    
    Returns:
        np.ndarray: Boolean array indicating segmented regions
    """
    assert im.dtype == np.uint8
    
    # Step 1: Initialize the segmented mask as a boolean array of the same shape as `im`
    segmented = np.zeros_like(im, dtype=bool)
    im = im.astype(float)  # Convert image to float for intensity calculations
    
    # Step 2: Define the Moore neighborhood (8-connected neighbors)
    # The neighbors relative to the current pixel position
    neighbors = [(-1, -1), (-1, 0), (-1, 1),
                 (0, -1),         (0, 1),
                 (1, -1), (1, 0), (1, 1)]
    
    # Step 3: Iterate over each seed point to grow regions
    for row, col in seed_points:
        # Initialize a queue with the seed point
        queue = [(row, col)]
        # Store the intensity of the seed point to compare with neighbors
        seed_intensity = im[row, col]
        
        # Set the seed point as part of the segmented region
        segmented[row, col] = True
        
        # Step 4: Region growing using a queue for candidate pixels
        while queue:
            current_row, current_col = queue.pop(0)
            
            # Check each of the 8 neighbors
            for d_row, d_col in neighbors:
                neighbor_row, neighbor_col = current_row + d_row, current_col + d_col
                
                # Check if the neighbor is within image bounds
                if (0 <= neighbor_row < im.shape[0]) and (0 <= neighbor_col < im.shape[1]):
                    # If neighbor has not been segmented and meets the threshold condition
                    if not segmented[neighbor_row, neighbor_col]:
                        intensity_diff = abs(im[neighbor_row, neighbor_col] - seed_intensity)
                        if intensity_diff <= T:
                            # Add neighbor to the region and to the queue
                            segmented[neighbor_row, neighbor_col] = True
                            queue.append((neighbor_row, neighbor_col))
    
    return segmented

In [3]:
if __name__ == "__main__":
    # DO NOT CHANGE
    im = utils.read_image("defective-weld.png")

    seed_points = [ # (row, column)
        [254, 138], # Seed point 1
        [253, 296], # Seed point 2
        [233, 436], # Seed point 3
        [232, 417], # Seed point 4
    ]
    intensity_threshold = 50
    segmented_image = region_growing(im, seed_points, intensity_threshold)

    assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
        im.shape, segmented_image.shape)
    assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
            segmented_image.dtype)

    segmented_image = utils.to_uint8(segmented_image)
    utils.save_im("defective-weld-segmented.png", segmented_image)


Reading image: images\defective-weld.png
Saving image to: image_processed\defective-weld-segmented.png


c:\Users\Bex\OneDrive - NTNU\NTNU\4 år\Visual dataprocessing\Assignment 6_ip\utils.py:56: UserWarning: C:\Users\Bex\OneDrive - NTNU\NTNU\4 år\Visual dataprocessing\Assignment 6_ip\image_processed\defective-weld-segmented.png is a low contrast image
  skimage.io.imsave(impath, im)
